<a href="https://colab.research.google.com/github/ikarlin86/projects/blob/main/recomendacao_filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Recomendação de Filmes***

In [1]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 14.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630181 sha256=22b9ff4158e59fa4c811cc8e9fef5c931af75093e22e1cb4a901ec7628c5bda6
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Reader, Dataset
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise import accuracy

In [3]:
# Dataset com o detalhe dos Filmes
movies = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/movies.csv')
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [4]:
# Dataset com as avaliações
ratings = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/recomenda_filmes/main/data/ratings.csv')
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [5]:
# Juntando as informações de filmes e avaliações
filmes = ratings.join(movies.set_index('movieId'), on='movieId')

In [6]:
print('Tamanho da Base de Filmes: ', filmes.shape)

Tamanho da Base de Filmes:  (100836, 6)


In [7]:
filmes.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [8]:
# Números dos datasets
print('Quantidade de Filmes Avaliados: ',
filmes['movieId'].value_counts().shape[0])

print('Quantidade de Usuários Avaliando: ',
filmes['userId'].value_counts().shape[0])

print('Quantidade de Avaliações: ',
ratings.shape[0])

Quantidade de Filmes Avaliados:  9724
Quantidade de Usuários Avaliando:  610
Quantidade de Avaliações:  100836


In [9]:
# Quantidade de Avaliações TOP5 Filmes
filmes['title'].value_counts().head()

Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: title, dtype: int64

In [10]:
# Quantidade de Avaliações LOW5 Filmes
filmes['title'].value_counts().tail()

Sex, Drugs & Taxation (2013)     1
Extraordinary Tales (2015)       1
Tomorrow (2015)                  1
Embrace of the Serpent (2016)    1
31 (2016)                        1
Name: title, dtype: int64

In [11]:
# Quantidade de Avaliações TOP5 Usuários
filmes['userId'].value_counts().head()

414    2698
599    2478
474    2108
448    1864
274    1346
Name: userId, dtype: int64

In [12]:
# Quantidade de Avaliações LOW5 Usuários
filmes['userId'].value_counts().tail()

442    20
569    20
320    20
576    20
53     20
Name: userId, dtype: int64

In [13]:
# Avaliações do usuário 414
filmes.query('userId == 274').head()

,userId,movieId,rating,timestamp,title,genres
39229,274,1,4.0,1171410158,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
39230,274,2,3.5,1171934785,Jumanji (1995),Adventure|Children|Fantasy
39231,274,6,4.0,1197022122,Heat (1995),Action|Crime|Thriller
39232,274,8,3.0,1172030892,Tom and Huck (1995),Adventure|Children
39233,274,10,4.0,1171428459,GoldenEye (1995),Action|Adventure|Thriller


In [14]:
# Avaliações do usuário 414
filmes.query('userId == 274 and movieId == 1')

,userId,movieId,rating,timestamp,title,genres
39229,274,1,4.0,1171410158,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [15]:
# Configuração para treinamento
reader = Reader(rating_scale=(0,5))
# Seleção das variáveis para o modelo
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# Divisão dos dados de treino e teste
trainset, testset = train_test_split(data, test_size=.25, random_state=42)

In [16]:
# Configurações das medidas de similaridade
sim_options = { 'name': 'pearson_baseline', 'user_based': True }

In [17]:
# Criação do modelo
knn = KNNBaseline(k=33, sim_options=sim_options)
# Treinamento do modelo
knn.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [18]:
# Criação do modelo
slo = SlopeOne()
# Treinamento do modelo
slo.fit(trainset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [19]:
# Criação do modelo
co = CoClustering(n_epochs=10, verbose=True, random_state=42)
# Treinamento do modelo
co.fit(trainset)

Processing epoch 0
Processing epoch 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [20]:
predictions_knn = knn.test(testset)
accuracy.rmse(predictions_knn)

RMSE: 0.8888


0.8887723564425568

In [21]:
predictions_slo = slo.test(testset)
accuracy.rmse(predictions_slo)

RMSE: 0.9134


0.9134291817129228

In [22]:
predictions_co = co.test(testset)
accuracy.rmse(predictions_co)

RMSE: 0.9534


0.953389277913654

In [23]:
# Predição com os dados de teste
predictions_knn[:5]

[Prediction(uid=50, iid=4282, r_ui=3.5, est=2.6893968587584185, details={'was_impossible': False}),
 Prediction(uid=603, iid=2993, r_ui=3.0, est=3.1298324602016714, details={'actual_k': 5, 'was_impossible': False}),
 Prediction(uid=140, iid=11, r_ui=4.0, est=3.680405802104295, details={'actual_k': 32, 'was_impossible': False}),
 Prediction(uid=262, iid=497, r_ui=4.0, est=3.506174818491485, details={'actual_k': 12, 'was_impossible': False}),
 Prediction(uid=492, iid=1363, r_ui=4.0, est=3.982283055858682, details={'actual_k': 0, 'was_impossible': False})]

In [25]:
# uid=247, iid=364 Conversão para o id interno
print('Conversão Interna')
print('User:', trainset.to_inner_uid(274))
print('Movie:', trainset.to_inner_iid(1))

# uid=247, iid=364 Conversão para o id externo
print('Conversão Externa')
print('User:', trainset.to_raw_uid( trainset.to_inner_uid(274) ))
print('Movie:', trainset.to_raw_iid( trainset.to_inner_iid(1) ))

Conversão Interna
User: 16
Movie: 287
Conversão Externa
User: 274
Movie: 1


In [26]:
def recomenda_filme(userId,movieId):
  # ID do usuário para predição
  uid = userId
  # ID do filme para predição
  iid = movieId 
  nome_filme = movies.query('movieId == @movieId')['title'].values[0]
  print('Filme:', nome_filme)
  print('Usuário:', userId)
  if filmes.query('userId == @userId and movieId == @movieId')['title'].values.size == 0:
    print('Usuário não avaliou o filme!')
  else:
    nota_filme = ratings.query('userId == @userId and movieId == @movieId')['rating'].values[0]
    print('Avaliação do usuário:', nota_filme)
  # Predição baseada no melhor modelo
  print('Estimativa de Avaliação[0-5]:', round(knn.predict(trainset.to_raw_uid(uid), trainset.to_raw_iid(iid))[3], 2))

In [27]:
recomenda_filme(406,1)

Filme: Toy Story (1995)
Usuário: 406
Usuário não avaliou o filme!
Estimativa de Avaliação[0-5]: 3.88


In [28]:
recomenda_filme(274,1) 


Filme: Toy Story (1995)
Usuário: 274
Avaliação do usuário: 4.0
Estimativa de Avaliação[0-5]: 3.32


In [29]:
def top_n(userId,n):
  # Selecionando apenas os filmes do treinamento
  lista_filmes_treino = []
  for x in trainset.all_items():
    lista_filmes_treino.append(trainset.to_raw_iid(x))
  # Selecionando os filmes do treinamento que o usuário não avaliou
  filmes_user = ratings.query('userId == @userId')['movieId'].values
  filmes_user_nao = movies.query('movieId not in @filmes_user')
  filmes_user_nao = filmes_user_nao.query('movieId in @lista_filmes_treino')['movieId'].values
  # Criando um ranking para o usuário para os filmes não avaliados
  ranking=[]
  for movieId in filmes_user_nao:
    ranking.append((movieId, knn.predict(trainset.to_inner_uid(userId), trainset.to_inner_iid(movieId))[3]))
  # Ordenando os TOP filmes avaliados
  ranking.sort(key=lambda x: x[1], reverse=True)
  # Selecionando os Ids dos filmes
  x,_ = zip(*ranking[:n])
  # Listando os nomes dos filmes em ordem de recomendação
  return movies.query('movieId in @x')['title'].copy().reset_index(drop=True)

In [30]:
top_n(274,5)


0                         It's My Party (1996)
1                    Mark of Zorro, The (1940)
2                          Dead Ringers (1988)
3                          Chuck & Buck (2000)
4    Gross Anatomy (a.k.a. A Cut Above) (1989)
Name: title, dtype: object

In [31]:
top_n(406,5)


0          Get Shorty (1995)
1    Mighty Aphrodite (1995)
2          Waterworld (1995)
3      Before Sunrise (1995)
4          Rent-a-Kid (1995)
Name: title, dtype: object